In [1]:
import os
import sys
import argparse
import torch
import random
import torch.nn.functional as F

In [2]:
os.chdir("/home/tslab/lab/21_story/tripod_dramatic/comet-commonsense")
sys.path.append("~/lab/21_story/tripod_dramatic/comet-commonsense")
print(os.getcwd())
import src.data.data as data
import src.data.config as cfg
import src.interactive.functions as interactive
import src.models.utils as model_utils

/home/tslab/lab/21_story/tripod_dramatic/comet-commonsense


In [3]:
prefixes = ['tries', 'starts', 'wants', 'begins', ]
random.seed(0)

In [4]:
def query(input_event, category='xNeed', sampling_algorithm='beam-10 '):
    sampler = interactive.set_sampler(opt, sampling_algorithm, data_loader)

    outputs = interactive.get_atomic_sequence(input_event, model, sampler, data_loader, text_encoder, category)[category]
    for k, v in outputs.items():
        outputs[k] = v[::-1]
    return outputs
    

In [5]:
#ARGS
device=0
model_file="/home/tslab/lab/comet_pretrained_models/atomic_pretrained_model.pickle"


In [6]:
#Load
opt, state_dict = interactive.load_model_file(model_file)
data_loader, text_encoder = interactive.load_data("atomic", opt)
n_ctx = data_loader.max_event + data_loader.max_effect
n_vocab = len(text_encoder.encoder) + n_ctx
model = interactive.make_model(opt, n_vocab, n_ctx, state_dict)
model.eval()

Loading data from: data/atomic/processed/generation/categories_oEffect#oReact#oWant#xAttr#xEffect#xIntent#xNeed#xReact#xWant-maxe1_17-maxe2_35-maxr_1.pickle
52


LMModel(
  (transformer): TransformerModel(
    (embed): Embedding(40542, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_1): LayerNorm()
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm()
      )
      (1): Block(
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_1): LayerNorm()
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm()
      )
    

In [15]:
#Inference Examples
category="oReact"
category="oWant"
character="[MALE]"
story="[MALE] had a big crush on [FEMALE]."
story="[MALE] took [FEMALE] fishing in summer."

# category="xAttr"
category="xIntent"
# character="[FEMALE]"
# story="[FEMALE] wanted to go to prom with him."
story="[FEMALE] was loving it."
with torch.no_grad():
    #Sample
    cfg.device = int(device)
    cfg.do_gpu = True
    torch.cuda.set_device(cfg.device)
    model.cuda(cfg.device)


    #Inference
    sampler = interactive.set_sampler(opt, "beam-10", data_loader)
    outputs = interactive.get_atomic_sequence(story, model, sampler, data_loader, text_encoder, category)[category]

    for k, v in outputs.items():
        outputs[k] = v[::-1]

    #3 Outputs
    reqs=outputs['beams'][:3]
    reqs = [[character, random.choice(prefixes)] + r.split() for r in reqs]
    print(f"reqs: {reqs}")


Input Event:   [MALE] took [FEMALE] fishing in summer.
Target Effect: oWant

Candidate Sequences:
none
to catch fish
to spend time with personx
to have fun
to catch a fish
to have fun with personx
to thank personx
to catch some fish
to go fishing
to thank personx for the catch


reqs: [['[MALE]', 'wants', 'to', 'thank', 'personx', 'for', 'the', 'catch'], ['[MALE]', 'starts', 'to', 'go', 'fishing'], ['[MALE]', 'begins', 'to', 'catch', 'some', 'fish']]
